In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
#df_gscore = pd.read_csv('data/genome-scores.csv')
#df_gtags = pd.read_csv('data/genome-tags.csv')
#df_links = pd.read_csv('data/links.csv')
#df_movies = pd.read_csv('data/movies.csv')
df_rating = pd.read_csv('data/ratings.csv')
#df_tagspd = pd.read_csv('data/tags.csv')

In [44]:
import pandas as pd
def cold_starters(amount = 10):
    #make a df from the rating database, grouped by movie Id with average rating and amount of rating
    df_rating = pd.read_csv('data/ratings.csv').groupby('movieId').agg({'rating': ['mean', 'count']})
    # Flatten the multi-index columns to have a single level of columns
    df_rating.columns = ['avg_rating', 'rating_count']
    #filtering out low view low rating movies
    df_rating = df_rating[(df_rating.avg_rating > 4) & (df_rating.rating_count > df_rating.rating_count.quantile(0.98))]
    # function to normalize data
    def Min_Max(obj):
        norm_obj =  (obj - obj.min()) / (obj.max() - obj.min())
        return norm_obj
    #normalize
    df_rating.avg_rating= Min_Max(df_rating.avg_rating)
    df_rating.rating_count= Min_Max(df_rating.rating_count)

    # getting the source dataframe
    from data_script.Preprocess_Content_Based import get_df
    df = get_df()
    df= pd.merge(df, df_rating, left_index=True, right_index=True)
    df = df[df.releaseyear > df.releaseyear.quantile(0.90)]
    df['score'] = df.avg_rating + 1.25 * df.rating_count - 1.5 * df.releaseyear
    #searching the movies in the dataframe and return
    df_movies = pd.read_csv('data/movies.csv')
    return df_movies[df_movies['movieId'].isin(df.sort_values(by='score').head(amount-1).index)]
    
cold_starters()

c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


,movieId,title,genres
7041,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
7249,7361,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi
11354,48394,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Drama|Fantasy|Thriller
11362,48516,"Departed, The (2006)",Crime|Drama|Thriller
11401,48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
12525,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
12746,60069,WALL·E (2008),Adventure|Animation|Children|Romance|Sci-Fi
13647,68157,Inglourious Basterds (2009),Action|Drama|War
13767,68954,Up (2009),Adventure|Animation|Children|Drama
15534,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX


In [5]:
# Creating the genome dataset

df_gtagscore = df_gtags.merge(df_gscore, how='right', on='tagId').drop('tag', axis=1)
df_gtagscore.drop_duplicates(subset=['tagId', 'movieId'], inplace=True)

# pivoting the dataframe
df_gtagscore = df_gtagscore.pivot(index='movieId', columns='tagId', values='relevance')

# adding release years as columns
def condition(x):
    if x[-2:] == 'a)':
        return np.nan
    elif x[-2:] == 'l)':
        return np.nan
    elif x[-3:-1] == '7-':
        return 2007
    elif x[-4:-2] == '9–':
        return 2009
    elif x[-2:] == '))':
        return x[-6:-2]
    elif x[-1:] == ')':
        return x[-5:-1]
    elif x[-1:] == ' ':
        return x[-6:-2]
    else:
        return np.nan

df_movies['releaseyear'] = df_movies['title'].apply(condition).fillna(1993)
df_movies = df_movies.drop('title', axis=1)
df_ContBaseRec = pd.merge(df_gtagscore, df_movies, how='inner', on='movieId').set_index('movieId')
df_ContBaseRec.columns = df_ContBaseRec.columns.astype(str)
df_ContBaseRec

,1,2,3,4,5,6,7,8,9,10,...,1121,1122,1123,1124,1125,1126,1127,1128,genres,releaseyear
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300,Adventure|Animation|Children|Comedy|Fantasy,1995
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875,Adventure|Children|Fantasy,1995
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850,Comedy|Romance,1995
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300,Comedy|Drama|Romance,1995
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825,Comedy,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130578,0.15400,0.04025,0.01975,0.01850,0.02850,0.02275,0.01000,0.03675,0.01025,0.01575,...,0.01250,0.02050,0.21000,0.02375,0.04225,0.00525,0.07575,0.01325,Action|Thriller,2015
130840,0.02400,0.03025,0.02425,0.09375,0.05600,0.06025,0.02100,0.12900,0.11275,0.03650,...,0.01025,0.20350,0.47675,0.10775,0.06200,0.00775,0.27000,0.13375,Horror|Romance|Sci-Fi,2015
131013,0.04200,0.03175,0.01700,0.05600,0.05750,0.02675,0.01600,0.04200,0.02800,0.03725,...,0.01275,0.02275,0.20000,0.10750,0.02500,0.00950,0.07550,0.01625,Comedy,2015


# recommending movies based on several movies

In [2]:
from model.ContentBasedRec_02 import recommendation


given_movies = [['Akira', '1988'], ['Network', '1976'], ['Prisoners', '2013'], ['Incendies', '2010'],
                ['red shoes', '1948'], ['metropolis', '1927']]

recommendation(given_movies, 20, 18)


c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)
c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


,labels,title,genres
movieId,,,
108981,15,Nymphomaniac: Volume II (2013),Drama|Mystery
37955,15,Innocence (2004),Drama|Fantasy|Mystery
1097,3,E.T. the Extra-Terrestrial (1982),Children|Drama|Sci-Fi
5473,15,Fox and His Friends (Faustrecht der Freiheit) ...,Drama
26599,15,"Law of Desire (Ley del deseo, La) (1987)",Comedy|Drama|Romance
6123,15,Sunless (Sans Soleil) (1983),Documentary
7172,15,Distant (Uzak) (2002),Drama
50619,15,"Blood of a Poet, The (Sang d'un poète, Le) (1930)",Drama|Fantasy
26231,15,Performance (1970),Crime|Drama|Thriller


# recommending movies based on a list of movie_Ids

In [3]:
import pandas as pd
sviatlana_rating = pd.read_csv('data/Sviatlana_ratings.csv')
sviatlana_rating
from model.ContentBasedRec_02 import recommendation
recommendation(sviatlana_rating['movie_id'].tolist() , 20, 18)

c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


,labels,title,genres
movieId,,,
94018,7,Battleship (2012),Action|Sci-Fi|Thriller|IMAX
39421,8,After School Special (a.k.a. Barely Legal) (2003),Comedy
98203,1,"Twilight Saga: Breaking Dawn - Part 2, The (2012)",Adventure|Drama|Fantasy|Romance|IMAX
631,17,All Dogs Go to Heaven 2 (1996),Adventure|Animation|Children|Fantasy|Musical|R...
92264,11,One for the Money (2012),Action|Comedy|Crime
32296,8,Miss Congeniality 2: Armed and Fabulous (2005),Adventure|Comedy|Crime
3687,11,Light Years (Gandahar) (1988),Adventure|Animation|Fantasy|Sci-Fi
53464,1,Fantastic Four: Rise of the Silver Surfer (2007),Action|Adventure|Sci-Fi
40962,8,"Yours, Mine and Ours (2005)",Comedy|Romance


In [1]:
import pandas as pd
hamid_rating = pd.read_csv('data/Hamid-ratings.csv')
from model.ContentBasedRec import recommendation
recommendation(hamid_rating['movie_id'].tolist() , 20, 18 , lambda_val = 0)

c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)
c:\Users\HamidEsfahlani\anaconda3\envs\MovieRecommend\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres,score
movieId,,,,
104069,14,Louis C.K.: Oh My God (2013),Comedy,1.000000
74152,14,Zach Galifianakis: Live at the Purple Onion (2...,Comedy|Documentary,0.987746
7620,12,Monster in a Box (1992),Comedy|Drama,0.985123
5847,13,Ragtime (1981),Drama,0.983882
1633,12,Ulee's Gold (1997),Drama,0.983640
127098,14,Louis C.K.: Live at The Comedy Store (2015),Comedy,0.982437
26712,6,35 Up (1991),Documentary,0.970934
86347,14,Louis C.K.: Chewed Up (2008),Comedy,0.963815
86345,14,Louis C.K.: Hilarious (2010),Comedy,0.954957


In [5]:
hamid_rating[hamid_rating['rating'] > 4].sample(n=20)

,movie_id,imdb_id,tmdb_id,rating,average_rating,title
27,593,102926,274,5.0,4.14954,The Silence of the Lambs (1991)
108,4262,86250,111,5.0,3.93819,Scarface (1983)
235,96606,770802,89708,4.5,4.06851,Samsara (2011)
82,2726,49406,247,4.5,4.01312,The Killing (1956)
62,1672,119978,11975,4.5,3.61454,The Rainmaker (1997)
17,306,111495,110,5.0,4.05459,Three Colors: Red (1994)
245,106768,1426374,39378,5.0,4.00000,The Wind Journeys (2009)
125,5878,287467,64,4.5,3.95518,Talk to Her (2002)
75,2571,133093,603,5.0,4.16019,The Matrix (1999)
44,1208,78788,28,5.0,4.11721,Apocalypse Now (1979)


In [3]:
#from model.HybridRecommendationSystem import hybrid_recommendation
sviatlana_rating = pd.read_csv('data/Sviatlana_ratings.csv')
sviatlana_rating
#hybrid_recommendation(,sviatlana_rating['movie_id'].tolist() )

,movie_id,imdb_id,tmdb_id,rating,average_rating,title
0,50,114814,629,5.0,4.26568,The Usual Suspects (1995)
1,111,75314,103,5.0,4.08056,Taxi Driver (1976)
2,293,110413,101,4.5,4.09683,Léon: The Professional (1994)
3,296,110912,680,5.0,4.19163,Pulp Fiction (1994)
4,307,108394,108,4.0,3.96434,Three Colors: Blue (1993)
...,...,...,...,...,...,...
121,201646,8772262,530385,4.5,3.79511,Midsommar (2019)
122,202439,6751668,496243,5.0,4.33089,Parasite (2019)
123,277066,7322224,497828,5.0,3.69317,Triangle of Sadness (2022)
124,278578,1016150,49046,5.0,3.81387,All Quiet on the Western Front (2022)


In [13]:
import numpy as np
def recommendation(given_movies, number_of_recommendations, num_clusters, lambda_val=np.random.rand()):
    # get the database
    from data_script.Preprocess_Content_Based import get_df
    df = get_df()
    # clustering the data
    from data_script.Clustering import get_labeledMovies
    df_labeled = get_labeledMovies(df, num_clusters)
    # checking if given_movies is an item of class list
    if isinstance(given_movies, list):
        # checking if given_movies is a list of lists
        if all(isinstance(item, list) for item in given_movies):
        # checking if it fits the [['movie Title_1','release year_1'],...,['movie Title_n','release year_n']] format.
            if all(len(item) == 2 and all(isinstance(sub_item, str) for sub_item in item) for item in given_movies):
                # convert the movies lists to list of integers cotnaining only Movie_Ids
                from features.search_movieId import search_movieId
                given_movies_ids = search_movieId(given_movies)
            else: 
                raise ValueError("Invalid format for given_movies. If it's a list of lists, it should contain lists of two strings [['movie_Title, release_year]].")
        # checking if the list consists only of integers
        elif all(isinstance(item, int) for item in given_movies):
            given_movies_ids = given_movies
        else: 
            raise ValueError("Invalid format for given_movies. If it's a list, it should contain lists or integers.")
    else:
            raise ValueError("Invalid format for given_movies. It should be a list.")
    
    # get rid of the moveis not represented in the preprocessed database
    given_movies_ids = [item for item in given_movies_ids if item in df.index.tolist()]
    
    # calculating the mean point (vector) of the given movies
    mean_point = df.loc[given_movies_ids].mean()

    # movie recommendation with cousin similarity
    from sklearn.metrics.pairwise import cosine_similarity
    # Get the relevance vector of the given movie
    given_movie_vector = mean_point.values.reshape(1, -1)
    # Calculate cosine similarity between the given movie and all other movies
    similarity_scores = cosine_similarity(df.values, given_movie_vector)
    # calculate the dissimilarity score
    dissimilarity_scores = similarity_scores - 1

    # distances as relevance score with nearest neighbors distances
    import sklearn.neighbors
    nn = sklearn.neighbors.NearestNeighbors(n_neighbors=df.shape[0])
    nn.fit(df)
    # getting the distances of any point in the df to mean point
    distances, indices = nn.kneighbors(mean_point.values.reshape(1, -1))
    # normalize the data between 0 and 1
    indices = (indices - indices.min()) / (indices.max() - indices.min())
    # Sort the distance according to the original index of the df
    relevance_score = np.argsort(indices[0])
    # calculate the relevance_score
    relevance_score = (1 - relevance_score)
    
    # Diversification
    score = pd.Series((similarity_scores[1, -1] + (lambda_val * dissimilarity_scores.reshape(1, -1)))[0], index=df.index, name="score")
    df_scored = pd.concat([df_labeled, score],axis=1)

    # Remove the given movie from the list of recommendation
    df_scored = df_scored.drop(given_movies_ids)
    # deleting the movies which are not in the same clusters as the given movies
    given_movies_clusters = df_labeled.loc[given_movies_ids, "labels"].unique()
    df_scored = df_scored[df_scored["labels"].isin(given_movies_clusters)]

    # getting the movies from the labled set which shows the title and label
    recommendations = df_scored.sort_values('score', ascending=False).head(number_of_recommendations)
    return recommendations 
    
hamid_rating = pd.read_csv('data/Hamid-ratings.csv')
recommendation(hamid_rating['movie_id'].tolist() , 20, 18, lambda_val=0)


c:\Users\HamidEsfahlani\Documents\GitHub\movie_recommendation_ProjectCDS\data_script\Preprocess_Content_Based.py:46: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)
c:\Users\HamidEsfahlani\anaconda3\envs\MovieRecommend\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,labels,title,genres,score
movieId,,,,
2,0,Jumanji (1995),Adventure|Children|Fantasy,0.659471
26148,4,"Train, The (1964)",Action|Thriller|War,0.659471
26122,7,Onibaba (1964),Drama|Horror|War,0.659471
26124,15,Robinson Crusoe on Mars (1964),Sci-Fi,0.659471
26131,4,"Battle of Algiers, The (La battaglia di Algeri...",Drama|War,0.659471
26133,0,"Charlie Brown Christmas, A (1965)",Animation|Children|Comedy,0.659471
26134,14,Dracula: Prince of Darkness (1966),Horror|Mystery,0.659471
26138,10,"Hill, The (1965)",Drama|War,0.659471
26139,4,In Harm's Way (1965),Drama|War,0.659471


In [ ]:
def HybridRanking(X, S, lambda_val, Sim1, Sim2):
    """
    Diversifies the ranking by applying the MMR (Maximal Marginal Relevance) technique.
    
    Args:
        X (list): List of items to be ranked.
        S (list): List of already selected items.
        lambda_val (float): Parameter to adjust the trade-off between relevance and diversity.
        Sim1 (function): Function to measure the relevance between an item and the user.
        Sim2 (function): Function to measure the similarity between two items.
    
    Returns:
        list: Diversified ranking of items.
    """
    max_marginal_relevance = -float('inf')
    omega_star = None
    
    for omega_i in X:
        if omega_i not in S:
            marginal_relevance = lambda_val * Sim1(omega_i) - (1 - lambda_val) * max(Sim2(omega_i, omega_j) for omega_j in S)
            if marginal_relevance > max_marginal_relevance:
                max_marginal_relevance = marginal_relevance
                omega_star = omega_i
    
    return omega_star


# Example usage:

# Define the relevance function Sim1 and similarity function Sim2
def Sim1(item):
    # Measure the relevance between the item and the user
    pass

def Sim2(item1, item2):
    # Measure the similarity between two items
    pass

# Set the input values
X = [...]  # List of items to be ranked
S = [...]  # List of already selected items
lambda_val = ...  # Parameter to adjust the trade-off between relevance and diversity

# Apply the diversification method
diversified_ranking = diversify_ranking(X, S, lambda_val, Sim1, Sim2)